# In-Context Learning

## 定义

1. GPT-2 预训练模型具有 zero-shot learning 能力
2. GPT-3 提出 In-context learning 概念， zero-shot 为 in-context learning 特例。其包含 zero/one/few-shot leanring 三种常见形式
3. in-context leanring 直观翻译是 “在上下文中学习”， “上下文“指输入信息。ICL 是 inference 技巧。
 
什么是 zero-shot learning？

1. 模型训练时未见过监督任务数据
2. 模型输入未有示例
3. learning 分两种，训练参数发生改变则是永久性的，而本 notebook 重点讨论**即时性学习**

给定概率模型:

$$
p_\theta( \text{output}|\text{input})
$$

在输入部分增加任务描述， 如翻译任务 `I have a dream.中译英:`, 其中`中译英:` 为任务描述，或者是一种提示或提示词。将输入描述和任务描述进行结构化组织的操作称之为格式化。

$$
p_\theta( \text{output}|\text{input}, \text{task})
$$

上述即为一种 zero-shot learning, 可以认为改变了“输入”分布，从而改变“输出”分布

格式化分为两种：

1. QA格式化：所有的问题都能表示为，“question：xxx answer：xxx”， 模型能理解问答模型
2. task-specified 格式化：“Q：小冬瓜 翻译-> A：”, “Q：context{我在东北玩泥巴}，rewrite->A:”

## 为什么预训练模型能 zero-shot leanring Work

1. 并非所有的模型都能 work， 如 BERT 或 一些参数量小的模型
2. language model 输入输出具有强灵活性，任意任务都能组织成文本序列（模版化）. 无法现象一个纯图像分类模型，在不改变模型结构和参数的情况下，zero-shot learning 去做图像检测任务。
3. 预训练数据中包含**知识**， 这种知识并非是具体的 task, 而是与 task 有关的内容。根据 GPT-2 论文 table.1 示例


”I’m not the cleverest man in the world, but like they say in
French: **Je ne suis pas un imbecile [I’m not a fool].**

In a now-deleted post from Aug. 16, Soheil Eid, Tory candidate
in the riding of Joliette, wrote in French: **”Mentez mentez,
il en restera toujours quelque chose,”** which translates as,
**”Lie lie and something will always remain.”**

“I hate the word **‘perfume'**" Burr says. ‘It’s somewhat better
in French: ‘parfum.’

If listened carefully at 29:55, a conversation can be heard
between two guys in French: “**-Comment on fait pour aller
de l’autre cote? -Quel autre cot ´ e?´** ”, which means “**-How
do you get to the other side? - What side?**”.

If this sounds like a bit of a stretch, consider this question in French: **As-tualler au cinema?**´ , or **Did you go to
the movies?**, which literally translates as Have-you to go to
movies/theater?

“**Brevet Sans Garantie Du Gouvernement**”, translated to
English: “**Patented without government warranty**”.

预训练数据中存在有类似“法语与英语翻译”的知识，相较与记忆知识，无监督预训练从海量语料中学会了 “翻译技巧”，**通用**学习了“文本推断逻辑”，这是预训练与监督学习的本质区别。

另外，zero-shot learning 也表明模型具有强**泛化性**。

## zero-shot 太硬核，那么 one-shot 和 few-shot 是否还有含金量？

什么是 one-shot 和 few-shot？ 在输入上增加示例：

$$
p_\theta( \text{output}|\text{input}, \text{task}, \text{example}_1, \text{example}_2, \ldots )
$$

该处理方法可以提供示例来约束问答格式，或者给到模型直接参考。所有的 in-context leanring 可以抽象成：

$$
p_\theta( \text{output}|\text{input}, \text{prompt}), 
$$

即 $\text{prompt}$ 是自由的，改变输入能提高回答的正确性，这类方法都具有含金量。

在涌现能力论文中，正是用 few-shot learning 方式测评，发现预训练模型部分结果好于一些经过“specified training” 的SOTA监督模型。

## zero-shot 是否是横空出世的？

1. zero-shot 一直是业界难点
2. GPT-2 在 Inference 阶段采用 zero-shot 方式在多个 NLP 任务上测评

需要综合 scaling-law， emergent-abilitis， in-context learning，unspervised-pretrained， transformer 等挂件因素，来考量 GPT-2/3 的影响力。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
torch.manual_seed(42)

## 创建语料和分词器

In [2]:

text = '''我有一个梦想,翻译摘要中文英文法文会,大语言模型,小冬瓜, i have a dream, abcdefghijklmnopqrstuvwxyzQWERTYUIOPASDFGHJKLZXCVBNM,~!@#$%^&*()_+`1234567890-={}[]:";'<>?,./\\\n'''
class SimplestTokenizer:
    def __init__(self, text):
        tokens = list(text)
        self.vocab = {}
        self.vocab_reverse = {}
        idx = 0
        for i in tokens:
            if i not in self.vocab:
                self.vocab[i] = idx
                self.vocab_reverse[idx] = i
                idx += 1
    def encode(self, text, return_pt = False):
        tokens = list(text)
        token_ids = [ self.vocab[token] for token in tokens]
        if return_pt:
            token_ids = torch.tensor(token_ids, dtype = torch.long).unsqueeze(0)
        return token_ids
    def decode(self, ids):
        token_list = [self.vocab_reverse[idx] for idx in ids]
        text = ''.join(token_list)
        return text
        
        
tokenizer = SimplestTokenizer(text)
print(tokenizer.encode('我要翻译中文'))
print(tokenizer.decode([0, 10, 7, 8, 11, 12]))

[0, 10, 7, 8, 11, 12]
我要翻译中文


## 创建模型

In [3]:
class InContextLearner(nn.Module):
    def __init__(self, vocab_size = 100):
        super().__init__()
        self.dim = 512
        self.embd = nn.Embedding(vocab_size, self.dim)
        self.w0 = nn.Linear(self.dim, self.dim)
        self.w1 = nn.Linear(self.dim, self.dim)
        self.lm_head = nn.Linear(self.dim, vocab_size)
    def forward(self, x):
        e = self.embd(x)
        e = torch.sin(self.w0(e).mean(dim = 1, keepdim=True)) + self.w1(e)
        logits = self.lm_head(e)
        return logits
        
vocab_size = len(tokenizer.vocab)
model = InContextLearner(vocab_size = vocab_size)
x = torch.randint(vocab_size, (1, 32))
model(x)

tensor([[[ 0.4684,  0.0700, -0.4173,  ...,  0.4209, -0.2419, -0.0283],
         [ 0.2404, -0.0940,  0.0741,  ...,  0.3407,  0.3816,  0.2700],
         [ 0.0018,  0.1066, -0.0899,  ..., -0.1597, -0.1091,  0.0417],
         ...,
         [ 0.7019,  0.2580, -0.6060,  ..., -0.5172,  0.1008, -0.1332],
         [ 0.0083,  0.2066, -0.2648,  ...,  0.1222,  0.5677,  0.0291],
         [ 0.3804,  0.2200, -0.6631,  ..., -0.2236, -0.4892,  0.3182]]],
       grad_fn=<ViewBackward0>)

## 推断

In [4]:
def generate(model, x, max_new_tokens = 20):
    for i in range(max_new_tokens):
        logits = model(x)
        new_token = torch.argmax(logits[:, -1, :], dim = -1)
        x = torch.cat((x, new_token.unsqueeze(1)), dim = 1)
    return x

text = '我有一个梦想'
input_ids = tokenizer.encode(text, return_pt = True)
result = generate(model, input_ids)[0,:].tolist()
text = tokenizer.decode(result)
print(text)

我有一个梦想f!AhcZ英C语O^梦D想语O^梦D想


## Zero-Shot

In [5]:
def format_translate(text):
    return '中文{' + text + '} 中译英> '

def forward_QA(text):
    return '\n#Question:' + text + '\n#Answer:'

text = '我有一个梦想'

text_task = format_translate(text)
text_task_qa = forward_QA(text_task)

input_ids = tokenizer.encode(text_task_qa, return_pt = True)
result = generate(model, input_ids)[0,:].tolist()
text = tokenizer.decode(result)
print(text)


#Question:中文{我有一个梦想} 中译英> 
#Answer:译D有型言J要 o&冬&冬&冬&冬&冬&


## One-Shot

In [6]:
def format_translate_example(text, output):
    return '中文{' + text + '} 中译英> ' + output +'\n'

example1 = format_translate_example('小模型', 'small model')

text_task_qa_1shot = example1 + text_task_qa

input_ids = tokenizer.encode(text_task_qa_1shot, return_pt = True)
result = generate(model, input_ids)[0,:].tolist()
text = tokenizer.decode(result)
print(text)

中文{小模型} 中译英> small model

#Question:中文{我有一个梦想} 中译英> 
#Answer:译D有型言J要 o&冬&冬&冬&冬&冬&


## Few-Shot

In [7]:

example2 = format_translate_example('小冬瓜', 'xiaodonggua AIGC')
example3 = format_translate_example('大语言模型', 'Large Language Model')

text_task_qa_fewshot = example1 + example2 + example3 + text_task_qa

input_ids = tokenizer.encode(text_task_qa_fewshot, return_pt = True)
result = generate(model, input_ids)[0,:].tolist()
text = tokenizer.decode(result)
print(text)

中文{小模型} 中译英> small model
中文{小冬瓜} 中译英> xiaodonggua AIGC
中文{大语言模型} 中译英> Large Language Model

#Question:中文{我有一个梦想} 中译英> 
#Answer:译D有型言J要 o)6*大"-型言J要 


# 总结

1. 上下文学习ICL 特点：a. 改变输入 b. 不改变参数 c. Inference 使用 4.即时学习
2. Training、Inference、ICL 他们推理形式是同源的；
3. Pretrained任务本身也是根据 context 进行推断
4. 表征学习视角理解 ICL ：few-shot 例子在 latent 层面**增加特定模式的推理特征**
5. 假设 context 是无限的，那么特定文本就一定能被生成出来（如随机模型根据随机context，产生一部莎士比亚的小说）